In [55]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader, SubsetRandomSampler
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

import os
import numpy as np
import matplotlib.pyplot as plt 

In [10]:
os.listdir('/kaggle/working/../input/labeled')

In [11]:
PROJECT_ROOT = '/kaggle/working/'

DATASET_PATH = os.path.join(PROJECT_ROOT, '../input/labeled')
CHECKPOINTS_PATH = os.path.join(PROJECT_ROOT, 'checkpoints')
PLOTS_PATH = os.path.join(CHECKPOINTS_PATH, 'plots')
os.makedirs(PLOTS_PATH, exist_ok=True)

In [12]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
          tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
          Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
          # The normalize code -> t.sub_(m).div_(s)
        return tensor

MEANS = [0.485, 0.456, 0.406]
STDS = [0.229, 0.224, 0.225]
unorm = UnNormalize(MEANS, STDS)

In [158]:
class MetricTracker:
    def __init__(self):
        self.batches_cnt = 0
        self.total_loss = 0
        self.avg_loss = 0
        
        self.confusion_cnt = {'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0}
        self.confusion_arrays = {'tp': [], 'tn': [], 'fp': [], 'fn': []}
        
        
    def update(self, loss, preds, labels):
        self.batches_cnt += 1
        
        self.total_loss += float(loss)
        self.avg_loss = self.total_loss / self.batches_cnt

        preds_sigmoid = torch.sigmoid(preds)
        preds_binary = preds_sigmoid > 0.5

        for i in range(len(labels)):        
            curr_key = ''
            if preds_binary[i] == 0 and labels[i] == 0:
                curr_key = 'tn'
            elif preds_binary[i] == 1 and labels[i] == 1:
                curr_key = 'tp'
            elif preds_binary[i] == 0 and labels[i] == 1:
                curr_key = 'fn'
            else:
                curr_key = 'fp'
                
            self.confusion_cnt[curr_key] += 1
            self.confusion_arrays[curr_key].append(float(preds_sigmoid[i]))
        
        
    def get_accuracy(self):
        confusion_cnt_sum = np.sum([cnt for _, cnt in self.confusion_cnt.items()])
        accuracy = self.confusion_cnt['tp'] + self.confusion_cnt['tn']
        accuracy /= confusion_cnt_sum
        
        return 100 * accuracy
    
    def get_precision(self):
        precision = self.confusion_cnt['tp'] / \
            (self.confusion_cnt['tp'] + self.confusion_cnt['fp'])
        
        return precision
    
    def get_recall(self):
        recall = self.confusion_cnt['tp'] / \
            (self.confusion_cnt['tp'] + self.confusion_cnt['fn'])
        
        return recall

In [152]:
def get_balanced_indices(dataset):

    positive_indices = [i for i in range(len(dataset.samples)) if dataset.samples[i][1] == 1]
    negative_indices = [i for i in range(len(dataset.samples)) if dataset.samples[i][1] == 0]
    negative_indices_balanced = np.random.choice(
        negative_indices, len(positive_indices), replace=False
        )
    
    return np.hstack([
        np.array(positive_indices), np.array(negative_indices_balanced)
        ])



def load_dataset_ECOL_labeled(
    root_dir, 
    img_size, 
    batch_size, 
    num_workers=0, 
    shuffle=True,
    balance=True,
    transform=None,
    dataset_size=None,
    test_size=0.2
    ):

    MEANS = [0.485, 0.456, 0.406]
    STDS = [0.229, 0.224, 0.225]

    if transform is None:
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(img_size),
            # transforms.Grayscale(num_output_channels=3),
            transforms.Normalize(mean=MEANS, std=STDS),
            ])

    dataset = ImageFolder(root_dir, transform=transform) 
    
    if dataset_size is None:
        dataset_size = len(dataset)
        
    if balance:
        dataset_indices = get_balanced_indices(dataset)
        
        if len(dataset_indices) < dataset_size:
            dataset_size = len(dataset_indices)
    else:
        dataset_indices = list(range(len(dataset)))
    
    
    if shuffle:
        dataset_indices = np.random.choice(
            dataset_indices, dataset_size, replace=False
            )
    else:
        dataset_indices = dataset_indices[ : dataset_size]
    
    val_split_index = int(np.floor(test_size * dataset_size))

    train_indices, val_indices = \
        dataset_indices[val_split_index:], dataset_indices[:val_split_index]

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(
        dataset=dataset, shuffle=False, batch_size=batch_size, sampler=train_sampler
        )
    val_loader = DataLoader(
        dataset=dataset, shuffle=False, batch_size=batch_size, sampler=val_sampler
        )


    return train_loader, val_loader

In [14]:
def show_batch_as_grid(input_batch, save_path=None):
    
    grid_img = torchvision.utils.make_grid(
        unorm(input_batch).cpu()
        )

    plt.figure(figsize=(6 * input_batch.shape[0], 10))
    plt.imshow(grid_img.permute(1, 2, 0))
    if save_path is not None:
        plt.savefig(save_path)
    plt.close()

    
def visualize_false_negatives(inputs, labels, preds, save_path=None):
    preds_binary = preds > 0.5
    is_false_negative = ((preds_binary == 0) * (labels == 1)).to(torch.bool).squeeze()
    
    if is_false_negative.sum() == 0:
        return 

    false_negatives = inputs[is_false_negative] 
    
    show_batch_as_grid(false_negatives, save_path)


def visualize_true_positives(inputs, labels, preds, save_path=None):
    preds_binary = preds > 0.5
    is_true_positive = ((preds_binary == 1) * (labels == 1)).to(torch.bool).squeeze()

    if is_true_positive.sum() == 0:
        return

    true_positives = inputs[is_true_positive]

    show_batch_as_grid(true_positives, save_path)

In [153]:
EPOCHS = 10
LEARNING_RATE = 0.0005
MAX_ITERS = 1000

LOADER_PARAMS = {
    'root_dir': DATASET_PATH, 'img_size': (256, 256), 'batch_size': 32, 
    'dataset_size': None, 'test_size': 0.2, 'balance': True
    }

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Loading the Data Loader
train_loader, val_loader = load_dataset_ECOL_labeled(**LOADER_PARAMS)

# Load the pretrained model
model = models.vgg16(pretrained=True)

# Replace the classification layer with the new one
# Will output the prediction for only 1 class
model.classifier[6] = nn.Linear(4096, 1)

# Move model to GPU
model = model.to(DEVICE)

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [157]:
def check_balancing(dataloader):
    positives_cnt = 0
    for ind in dataloader.sampler.indices:
        positives_cnt += dataloader.dataset.samples[ind][1]

    print(positives_cnt / len(dataloader.sampler.indices))
    
    
check_balancing(train_loader)
check_balancing(val_loader)

In [107]:
def train_epoch(model, dataloader, optimizer, epoch_idx=0):
    
    # Put the model in training mode
    model.train()
    
    # Initialize the metrics
    metric_tracker = MetricTracker()
    if 'sampler' in dir(dataloader):
        total_batch_num = len(dataloader.sampler.indices) // dataloader.batch_size
    else:
        total_batch_num = len(dataloader.dataset) // dataloader.batch_size
    
    for i, data in enumerate(dataloader):
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
        labels = labels.unsqueeze(dim=1).to(torch.float32) 
        
        # Reset the optimizer gradient
        optimizer.zero_grad()
        
        # Forward pass
        preds = model(inputs)
            
        loss = criterion(preds, labels)
        
        # Backwards pass and parameter update
        loss.backward()
        optimizer.step()
        
        # Update the metrics
        metric_tracker.update(float(loss), preds, labels)
        
        if i % 10 == 0:
            print(f'Train epoch {epoch_idx}, Batch {i}/{total_batch_num}: {float(loss)}')
    
    print(f'\n=== TRAIN - Epoch {epoch_idx} ===')
    print(f'Avg loss = {metric_tracker.avg_loss}')
    print(f'Accuracy = {metric_tracker.get_accuracy()}')
    print(f'Precision = {metric_tracker.get_precision()}')
    print(f'Recall = {metric_tracker.get_recall()}')
    print()
    
    return metric_tracker


def evaluate(model, dataloader):
    
    # Put the model in eval mode
    model.eval()
    
    # Initialize the metrics
    metric_tracker = MetricTracker()
    if 'sampler' in dir(dataloader):
        total_batch_num = len(dataloader.sampler.indices) // dataloader.batch_size
    else:
        total_batch_num = len(dataloader.dataset) // dataloader.batch_size
    
    for i, data in enumerate(dataloader):
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
        labels = labels.unsqueeze(dim=1).to(torch.float32) 
        
        # Forward pass
        preds = model(inputs)
            
        loss = criterion(preds, labels)
        
        # Update the metrics
        metric_tracker.update(float(loss), preds, labels)
        
        if i % 10 == 0:
            print(f'Validation - Batch {i}/{total_batch_num}: {float(loss)}')
    
    print(f'\n=== VALIDATION ===')
    print(f'Avg loss = {metric_tracker.avg_loss}')
    print(f'Accuracy = {metric_tracker.get_accuracy()}')
    print(f'Precision = {metric_tracker.get_precision()}')
    print(f'Recall = {metric_tracker.get_recall()}')
    print()
    
    return metric_tracker
    
    

In [159]:
metrics_per_epoch = {}

for epoch in range(10):
    metrics_per_epoch[epoch] = {}
    
    train_metrics = train_epoch(model, train_loader, optimizer, epoch)
    metrics_per_epoch[epoch]['train'] = train_metrics
    
    val_metrics = evaluate(model, val_loader)
    metrics_per_epoch[epoch]['val'] = train_metrics
    

In [ ]:
losses = []
accuracies = []

total_batch_num = len(dataloader.dataset) // dataloader.batch_size
# Training loop
for epoch in range(30):
    model.train()
    os.makedirs(os.path.join(PLOTS_PATH, f'epoch_{epoch}'), exist_ok=True)

    if i % 30 == 0:
        save_path = os.path.join(PLOTS_PATH, f'epoch_{epoch}', f'false_negative_batch_{i}.png')
        visualize_false_negatives(inputs, labels, preds_sigmoid, save_path)

        save_path = os.path.join(PLOTS_PATH, f'epoch_{epoch}', f'true_positive_batch_{i}.png')
        visualize_true_positives(inputs, labels, preds_sigmoid, save_path)


In [ ]:
plt.hist(TPp)
plt.xlim([0, 1])

In [ ]:
plt.hist(TNp)
plt.xlim([0, 1])

In [ ]:
plt.hist(FPp)
plt.xlim([0, 1])

In [ ]:
plt.hist(FNp)
plt.xlim([0, 1])